In [1]:
from typing import Literal
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import itertools

import torch
from torch import nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score

from fddbenchmark.dataset import FDDDataset
from fddbenchmark.dataloader import FDDDataloader
from fddbenchmark.models.detection.pca import FaultDetectionPCA
from fddbenchmark.models.detection.lstm_vae import FaultDetectionLSTMVAE

In [2]:
dataset = FDDDataset(name='small_tep')

Reading data/small_tep/labels.csv: 100%|████| 153300/153300 [00:00<00:00, 3070541.79it/s]
Reading data/small_tep/train_mask.csv: 100%|█| 153300/153300 [00:00<00:00, 3102095.30it/s
Reading data/small_tep/test_mask.csv: 100%|█| 153300/153300 [00:00<00:00, 3087897.90it/s]


# PCA

In [3]:
train_dataloader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.train_mask & (dataset.labels == 0),
    labels=dataset.labels,
    window_size=100,
    dilation=5,
    step_size=1,
)
test_dataloader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.test_mask,
    labels=dataset.labels,
    window_size=100,
    dilation=5,
    step_size=1,
)

Creating sequence of samples: 100%|██████████████████| 105/105 [00:00<00:00, 2381.71it/s]


In [7]:
pca = FaultDetectionPCA(normal_variance_ratio=0.95, scoring='spe')
pca.fit(train_dataloader)
(X, y, _), = iter(test_dataloader)
roc_auc_score(y > 0, pca.predict(X))

0.9176465731619414

# LSTM-based VAE

In [3]:
train_dataloader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.train_mask & (dataset.labels == 0),
    labels=dataset.labels,
    window_size=100,
    dilation=1,
    step_size=1,
    use_minibatches=True,
    batch_size=32,
    shuffle=True,
)

test_dataloader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.test_mask,
    labels=dataset.labels,
    window_size=100,
    dilation=1,
    step_size=1,
    use_minibatches=True,
    batch_size=32
)

Creating sequence of samples: 100%|██████████████████| 105/105 [00:00<00:00, 1750.99it/s]


In [6]:
lstm_vae = FaultDetectionLSTMVAE(
    input_dim=train_dataloader.df.shape[1],
    hidden_dim=128,
    latent_dim=8,
    scoring='importance_sampling'
)

In [9]:
lstm_vae._fit_scaler(train_dataloader)

In [5]:
lstm_vae.fit(
    train_dataloader,
    num_epochs=1000,
    log_dir='/homes/mgoncharov/summerschool2023-benchmark/logs/'
)

Epoch 999, training loop: 100%|██████████████████████████| 63/63 [00:01<00:00, 60.19it/s]


In [11]:
torch.save(lstm_vae.state_dict(), '/homes/mgoncharov/summerschool2023-benchmark/logs/version_1/ckpt.pt')

In [7]:
lstm_vae.load_state_dict(torch.load('/homes/mgoncharov/summerschool2023-benchmark/logs/version_1/ckpt.pt'))

<All keys matched successfully>

In [10]:
y_true = []
y_score = []
for (x, y, _) in tqdm(test_dataloader):
    y_true.append(y)
    y_score.append(lstm_vae.predict(x))

  0%|          | 0/2826 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 30.59 GiB (GPU 0; 15.73 GiB total capacity; 1.79 GiB already allocated; 2.68 GiB free; 1.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
roc_auc_score(np.concatenate(y_true) > 0, np.concatenate(y_score))

0.8937985223450264